In [1]:
import xarray as xr
from ocf_blosc2 import Blosc2
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
import json
with open("indices.json") as f:
    site_locations = {
        data_source: {
            int(site): (int(location[0]), int(location[1]))
            for site, location in locations.items()
        }
        for data_source, locations in json.load(f).items()
    }

In [3]:
from dataset import ChallengeDataset

In [4]:
ds = ChallengeDataset('/data/pv/2021/', '/data/satellite-nonhrv_proc/2021/', "/data/weather_proc/2021/", site_locations, None)
len(ds)

3305577

In [7]:
import sys
item = ds[0]
ctx = 0
for i in item:
    ctx += i.nbytes
print(ctx * len(ds))

16465845930864


In [ ]:
list(ds.open_parquet('/data/pv/2021/1.parquet')) == list(ds.open_parquet('/data/pv/2021/1.parquet').xs("2021-01-01 08:00:00").index)

In [ ]:
len(ds.open_parquet('/data/pv/2021/1.parquet').index)

In [11]:

results = []
for i in range(16):
    print(i)    
    results.append(ds[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [12]:
del results